In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Click Dots below for CSV Imports

In [9]:
# Import All of the CSVs

read_csv_16 = "Resources/2016_team_data.csv"

raw_data_16 = pd.read_csv(read_csv_16)

read_csv_17 = "Resources/2017_team_data.csv"

raw_data_17 = pd.read_csv(read_csv_17)

read_csv_18 = "Resources/2018_team_data.csv"

raw_data_18 = pd.read_csv(read_csv_18)

read_csv_19 = "Resources/2019_team_data.csv"

raw_data_19 = pd.read_csv(read_csv_19)

read_csv_20 = "Resources/2020_team_data.csv"

raw_data_20 = pd.read_csv(read_csv_20)

# I do not understand why there are NaaN values. 
# Still figuring out this whole database building thing. 

db_16 = raw_data_16.dropna()

db_17 = raw_data_17.dropna()

db_18 = raw_data_18.dropna()

db_19 = raw_data_19.dropna()

db_20 = raw_data_20.dropna()


In [6]:
# We Only Need Runs Scored & Runs Allowed
# Recall that 2020 was a shortened season -- I went ahead and created an "Assumed Variable" 
# based upon the average runs scored per game vs. 162 games. 
# I understand that this is a significant limitation to this dataset. 
# Suggestions on how to more accurately forecast this statistic are welcome.

In [23]:
# Creating New DB 

db_19.columns

Index(['Tm', '#Bat', 'BatAge', 'R/G', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B',
       'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS', 'OPS+',
       'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB', 'LOB', '#P', 'PAge', 'RA/G', 'W',
       'L', 'W-L%', 'ERA', 'G.1', 'GS', 'GF', 'CG', 'tSho', 'cSho', 'SV', 'IP',
       'H.1', 'R.1', 'ER', 'HR.1', 'BB.1', 'IBB.1', 'SO.1', 'HBP.1', 'BK',
       'WP', 'BF', 'ERA+', 'FIP', 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W',
       'LOB.1'],
      dtype='object')

In [25]:
rs_20 = db_20['Adj. Runs']
ra_20 = db_20['Adj. Runs.1']
teams = db_20['Tm']

rs_19 = db_19['R']
ra_19 = db_19['R.1']

rs_18 = db_18['R']
ra_18 = db_18['R.1']

rs_17 = db_17['R']
ra_17 = db_17['R.1']

rs_16 = db_16['R']
ra_16 = db_16['R.1']

In [31]:
df = pd.DataFrame({
    
    "Teams": teams,
    "Runs Scored '20": rs_20,
    "Runs Allowed '20": ra_20,
    "Runs Scored '19": rs_19,
    "Runs Allowed '19": ra_19,
    "Runs Scored '18": rs_18,
    "Runs Allowed '18": ra_18,
    "Runs Scored '17": rs_17,
    "Runs Allowed '17": ra_17,
    "Runs Allowed '16": ra_16,
    "Runs Scored '16": rs_16
    
})

df.head()

,Teams,Runs Scored '20,Runs Allowed '20,Runs Scored '19,Runs Allowed '19,Runs Scored '18,Runs Allowed '18,Runs Scored '17,Runs Allowed '17,Runs Allowed '16,Runs Scored '16
0,ARI,725.76,797.04,813.0,743.0,693.0,644.0,812.0,659.0,890.0,752.0
1,ATL,939.60,777.60,855.0,743.0,759.0,657.0,732.0,821.0,779.0,649.0
2,BAL,740.34,793.80,729.0,981.0,622.0,892.0,743.0,841.0,715.0,744.0
3,BOS,788.94,947.70,901.0,828.0,876.0,647.0,785.0,668.0,694.0,878.0
4,CHC,716.04,648.00,814.0,717.0,761.0,645.0,822.0,695.0,556.0,808.0


In [63]:
# Calculations

# Bill James' Pythag Win % is derived from (Runs Scored ^ 2 / [ Runs Scored^2 * Runs Allowed^2])

pywins16 = ((np.exp2(rs_16)) / ((np.exp2(rs_16))+(np.exp2(ra_16)))*100)

